In [10]:
import sys
import os
sys.path.append("../")

import pandas as pd
import json
import copy
from utils import text_util, file_util, excel_tree_level_export
from utils import graph_builder_from_wiki as graph_wiki_util
import crawl_wiki_tree

In [11]:
data_dir = "../data"
csv_path = os.path.join(data_dir, "reuters.csv")
name_mention_dir = os.path.join(data_dir, "test_reuters_entities")
file_type = "_entity_dict_iteration.pck"
output_prefix = os.path.join(data_dir, "reuters_all_entity")
max_parent_level = 3

In [12]:
df = pd.read_csv(csv_path)
df.head()

,path,topic,subset,index,content,lead,tin,retail,fuel,propane,...,soy-meal,earn,sun-oil,instal-debt,cotton,heat,trade,dfl,palladium,iron-steel
0,test/14826,['trade'],test,14826,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,test/14828,['grain'],test,14828,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,test/14829,"['nat-gas', 'crude']",test,14829,JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,test/14832,"['rubber', 'tin', 'sugar', 'corn', 'rice', 'gr...",test,14832,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,test/14833,"['palm-oil', 'veg-oil']",test,14833,INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df.shape

(10788, 95)

In [14]:
label_columns = []
for x in df.columns:
    if x not in ("path", "topic", "subset", "index", "content"):
        label_columns.append(x)

print("Number of labels:", len(label_columns))
print(label_columns)

Number of labels: 90
['lead', 'tin', 'retail', 'fuel', 'propane', 'crude', 'income', 'oat', 'copra-cake', 'barley', 'groundnut', 'cotton-oil', 'rand', 'cpi', 'lei', 'cocoa', 'groundnut-oil', 'jobs', 'nkr', 'livestock', 'castor-oil', 'palmkernel', 'money-fx', 'sunseed', 'hog', 'nat-gas', 'zinc', 'coconut-oil', 'gas', 'rape-oil', 'gold', 'orange', 'pet-chem', 'wheat', 'nickel', 'jet', 'interest', 'carcass', 'bop', 'l-cattle', 'potato', 'rapeseed', 'sugar', 'coffee', 'soy-oil', 'money-supply', 'platinum', 'yen', 'wpi', 'ship', 'soybean', 'sorghum', 'lin-oil', 'dmk', 'meal-feed', 'coconut', 'rice', 'dlr', 'alum', 'oilseed', 'acq', 'reserves', 'ipi', 'corn', 'grain', 'housing', 'nzdlr', 'naphtha', 'strategic-metal', 'palm-oil', 'sun-meal', 'lumber', 'tea', 'rye', 'rubber', 'gnp', 'veg-oil', 'cpu', 'silver', 'copper', 'soy-meal', 'earn', 'sun-oil', 'instal-debt', 'cotton', 'heat', 'trade', 'dfl', 'palladium', 'iron-steel']


In [15]:
def normalize(text):
    split_text = text.split("\n")
    subject = split_text.pop(0).strip()
    content = " ".join(x.strip() for x in split_text).strip()
    merge_text = "{} | {}".format(subject, content)
    return merge_text

normalize(df["content"][0])[:1000]

'ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT | Mounting trade friction between the U.S. And Japan has raised fears among many of Asia\'s exporting nations that the row could inflict far-reaching economic damage, businessmen and officials said. They told Reuter correspondents in Asian capitals a U.S. Move against Japan might boost protectionist sentiment in the U.S. And lead to curbs on American imports of their products. But some exporters said that while the conflict would hurt them in the long-run, in the short-term Tokyo\'s loss might be their gain. The U.S. Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17, in retaliation for Japan\'s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost. Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports of products hit by the new taxes. "We would

In [16]:
df["content"] = df["content"].apply(normalize)

### Extract name mentions from each text file

In [17]:
text_util.get_name_mention_from_claims_nltk([df["content"][0]])[:10]

['Washington',
 'Taiwan Safe',
 'new taxes',
 'partners',
 'protectionist sentiment',
 'complete erosion',
 'Japanese diplomatic manoeuvres',
 'Reuter correspondents',
 'Move',
 'Australian government']

In [18]:
file_util.mkdir(name_mention_dir)

for row_id, row in df.iterrows():
    doc_id = row["index"]
    doc_content = row["content"]
    doc_name_mentions = text_util.get_name_mention_from_claims_nltk([doc_content])
    # doc_ngram_name_mentions = doc_name_mentions
    doc_ngram_name_mentions = text_util.generate_n_gram_from_name_mentions(doc_name_mentions, 2)
    doc_file = open(name_mention_dir + "/name_mention_" + str(doc_id) + ".txt", "w")
    for x in doc_ngram_name_mentions:
        doc_file.write(x + "\n")
    doc_file.close()

In [30]:
1101 in df["index"].values

True

In [20]:
print(len(os.listdir(name_mention_dir)))

10788


In [9]:
crawl_wiki_tree.search_wiki_with_threads(name_mention_dir, 0, None, iteration=max_parent_level)

THREAD 0  START


../utils/wiki_util.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results_df = pd.io.json.json_normalize(results['results']['bindings'])


PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_iteration.pck
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_not_found_iteration.pck
0 / 158 )


../utils/wiki_util.py:98: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results_df = pd.io.json.json_normalize(results['results']['bindings'])
../utils/wiki_util.py:136: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results_df = pd.io.json.json_normalize(results['results']['bindings'])


PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_iteration.pck
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_not_found_iteration.pck
1 / 158 )
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_iteration.pck
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_not_found_iteration.pck
2 / 158 )
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_iteration.pck
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_not_found_iteration.pck
3 / 158 )
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_iteration.pck
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_not_found_iteration.pck
4 / 158 )
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_iteration.pck
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_not_found_iteration.pck
5 / 158 )
PATH ../data/reuters_entities/name_mention_14826.txt_entity_dict_iteration.pck
PATH ../data/reuters_entities/n

KeyboardInterrupt: 

In [10]:
graph_wiki_util.merge_crawled_data(name_mention_dir, file_type, output_prefix)

Merge crawled data: 100%|██████████| 3860/3860 [00:48<00:00, 80.16it/s] 


PATH ../data/reuters_all_entity.pck


In [4]:
crawl_wiki_tree.update_entity_details(name_mention_dir, file_type, output_prefix)

file_name ../data/reuters_entities/name_mention_842.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_8416.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_8415.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_8413.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_8409.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_8407.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_8406.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_8405.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_8402.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_8401.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_8400.txt_entity_dict_iteration.pck
file_name ../data/reuters_entities/name_mention_840.txt_entity_dict_iteration.pck
file_n

TypeError: 'NoneType' object is not iterable

In [5]:
graph_wiki_util.convert_to_tree(file_util.load_json(output_prefix + ".json"), file_util.load_json(output_prefix + "_brief.json"))

FileNotFoundError: [Errno 2] No such file or directory: '../data/reuters_all_entity_brief.json'

In [24]:
excel_tree_level_export.demo(file_util.load_json("all_entity_level.json"))